<a href="https://colab.research.google.com/github/ptmhoang97/train_dataset/blob/main/train_model_transferlearning_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 23 13:40:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [217]:
batch_size_pipeline = 4
num_steps_training = 3000
learning_rate_pipeline = 0.1
warmup_learning_rate_pipeline = learning_rate_pipeline/10
models_drive_path = "/content/drive/MyDrive/workspaces_tensorflow/test_continue_training"
train_name = 'test'
model_name = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
train_dataset = "/content/drive/MyDrive/train.zip"
test_dataset = "/content/drive/MyDrive/test.zip"

# 0. Setup Paths

In [224]:
import os

In [225]:
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [226]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc'),
    'MODEL_PATH': os.path.join(models_drive_path,train_name),
    'CHECKPOINT_PATH': os.path.join(models_drive_path, train_name, 'checkpoint'), 
    'OUTPUT_PATH': os.path.join(models_drive_path, train_name, 'export'), 
    'TFJS_PATH':os.path.join(models_drive_path, train_name, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join(models_drive_path, train_name, 'tfliteexport'), 
    'PRETRAINED_MODEL_PATH':os.path.join(models_drive_path, model_name), 
 }

In [227]:
files = {
    'PIPELINE_CONFIG':os.path.join(models_drive_path, train_name, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME),
    'PIPELINE_CONFIG_BASE':os.path.join(models_drive_path, model_name, 'pipeline.config'),
    'CHECKPOINT_BASE':os.path.join(models_drive_path, model_name, 'checkpoint'),
}

In [228]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}
        print(path)

# 1. Install TF Object Dectection API

In [10]:
# https://www.tensorflow.org/install/source_windows

In [11]:
if os.name=='nt':
    !pip install wget
    import wget

In [12]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 63156, done.
remote: Counting objects: 100% (768/768), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 63156 (delta 595), reused 749 (delta 581), pack-reused 62388
Receiving objects: 100% (63156/63156), 574.78 MiB | 33.72 MiB/s, done.
Resolving deltas: 100% (44050/44050), done.


In [13]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Processing /content/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 12.6 MB/s 
     |████████████████████████████████| 352 kB 64.1 MB/s 
     |████████████████████████████████| 1.8 MB 60.1 MB/s 
     |████████████████████████████████| 636 kB 59.8 MB/s 
     |████████████████████████████████| 99 kB 9.5 MB/s 
     |████████████████████████████████| 3

In [14]:
print(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py'))

Tensorflow/models/research/object_detection/builders/model_builder_tf2_test.py


In [15]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2021-09-23 13:47:42.777894: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 13:47:42.993110: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 13:47:42.993774: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-09-23 13:47:43.006762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021

In [16]:
import object_detection

# 2. Download dataset

In [17]:
!unzip {train_dataset} -d {paths['IMAGE_PATH']}

Archive:  /content/drive/MyDrive/train.zip
   creating: Tensorflow/workspace/images/train/
  inflating: Tensorflow/workspace/images/train/0013d97a44c586c0_jpg.rf.c129e0f75e464334397783e12eeddd69.jpg  
  inflating: Tensorflow/workspace/images/train/0013d97a44c586c0_jpg.rf.c129e0f75e464334397783e12eeddd69.xml  
  inflating: Tensorflow/workspace/images/train/0013d97a44c586c0_jpg.rf.QohOjYeZzQhUb4dxfAUr.jpg  
  inflating: Tensorflow/workspace/images/train/0013d97a44c586c0_jpg.rf.QohOjYeZzQhUb4dxfAUr.xml  
  inflating: Tensorflow/workspace/images/train/0053860714da4340_jpg.rf.08dfd541dc4fba8e821160c5989f18d7.jpg  
  inflating: Tensorflow/workspace/images/train/0053860714da4340_jpg.rf.08dfd541dc4fba8e821160c5989f18d7.xml  
  inflating: Tensorflow/workspace/images/train/0053860714da4340_jpg.rf.W5imfEtsa9yRvs4xbBiq.jpg  
  inflating: Tensorflow/workspace/images/train/0053860714da4340_jpg.rf.W5imfEtsa9yRvs4xbBiq.xml  
  inflating: Tensorflow/workspace/images/train/005980b3a5f93085_jpg.rf.2QZCXG

In [18]:
!unzip {test_dataset} -d {paths['IMAGE_PATH']}

Archive:  /content/drive/MyDrive/test.zip
   creating: Tensorflow/workspace/images/test/
  inflating: Tensorflow/workspace/images/test/00dea1edf14f09ab_jpg.rf.3f17c8790a68659d03b1939a59ccda80.jpg  
  inflating: Tensorflow/workspace/images/test/00dea1edf14f09ab_jpg.rf.3f17c8790a68659d03b1939a59ccda80.xml  
  inflating: Tensorflow/workspace/images/test/00dea1edf14f09ab_jpg.rf.KJ730oDTFPdXdJxvSLnX.jpg  
  inflating: Tensorflow/workspace/images/test/00dea1edf14f09ab_jpg.rf.KJ730oDTFPdXdJxvSLnX.xml  
  inflating: Tensorflow/workspace/images/test/00e481ea1a520175_jpg.rf.6e6a8b3b45c9a11d106958f88ff714ea.jpg  
  inflating: Tensorflow/workspace/images/test/00e481ea1a520175_jpg.rf.6e6a8b3b45c9a11d106958f88ff714ea.xml  
  inflating: Tensorflow/workspace/images/test/00e481ea1a520175_jpg.rf.MV6sZ8QCFwFeMYaI2tHm.jpg  
  inflating: Tensorflow/workspace/images/test/00e481ea1a520175_jpg.rf.MV6sZ8QCFwFeMYaI2tHm.xml  
  inflating: Tensorflow/workspace/images/test/08c8b73e0c2e296e_jpg.rf.7IkYAamjZhnwsoXSr

# 3. Create Label Map

In [19]:
labels = [
          {'name':'Ambulance', 'id':1},
          {'name':'Bus', 'id':2},
          {'name':'Car', 'id':3},
          {'name':'Motorcycle', 'id':4},
          {'name':'Truck', 'id':5},
          ]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 4. Create TF records

In [20]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/ptmhoang97/GenerateTFRecord_roboflow {paths['SCRIPTS_PATH']}

Cloning into 'Tensorflow/scripts'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [21]:
train_dataset_folder = train_dataset[train_dataset.rfind("/")+1:train_dataset.rfind(".")]
test_dataset_folder = test_dataset[test_dataset.rfind("/")+1:test_dataset.rfind(".")]
print(train_dataset_folder)
print(test_dataset_folder)

train
test


In [22]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], train_dataset_folder)} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], test_dataset_folder)} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 5. Copy Model Config to Training Folder

In [229]:
if os.name =='posix':
    !cp {os.path.join(files['PIPELINE_CONFIG_BASE'])} {os.path.join(paths['MODEL_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(files['PIPELINE_CONFIG_BASE'])} {os.path.join(paths['MODEL_PATH'])}

# 6. Update Config For Transfer Learning

In [233]:
# Get lastest base ckpt
import re
import numpy as np

lst = os.listdir(files['CHECKPOINT_BASE'])
lst = [l for l in lst if 'ckpt-' in l and '.index' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.index', '')
lastest_base_ckpt = os.path.join(files['CHECKPOINT_BASE'],last_model)
print(lastest_base_ckpt)

/content/drive/MyDrive/workspaces_tensorflow/test_continue_training/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0


In [234]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [235]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [236]:
config

{'eval_config': metrics_set: "coco_detection_metrics"
 use_moving_averages: false,
 'eval_input_config': label_map_path: "PATH_TO_BE_CONFIGURED"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "PATH_TO_BE_CONFIGURED"
 },
 'eval_input_configs': [label_map_path: "PATH_TO_BE_CONFIGURED"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "PATH_TO_BE_CONFIGURED"
 }
 ],
 'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.9969999790191

In [237]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [240]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = batch_size_pipeline
pipeline_config.train_config.num_steps = num_steps_training
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.total_steps = num_steps_training
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = learning_rate_pipeline
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = warmup_learning_rate_pipeline
pipeline_config.train_config.fine_tune_checkpoint = lastest_base_ckpt
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [241]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 7. Train the model

In [242]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [243]:
command = "python {} --model_dir={} --pipeline_config_path={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [244]:
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=/content/drive/MyDrive/workspaces_tensorflow/test_continue_training/test/checkpoint --pipeline_config_path=/content/drive/MyDrive/workspaces_tensorflow/test_continue_training/test/pipeline.config


In [245]:
!{command}

2021-09-23 16:31:28.794002: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 16:31:28.803205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 16:31:28.803827: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 16:31:28.805070: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 16:31:28.805712: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

# 8. Evaluate model

In [246]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [247]:
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=/content/drive/MyDrive/workspaces_tensorflow/test_continue_training/test/checkpoint --pipeline_config_path=/content/drive/MyDrive/workspaces_tensorflow/test_continue_training/test/pipeline.config --checkpoint_dir=/content/drive/MyDrive/workspaces_tensorflow/test_continue_training/test/checkpoint


In [248]:
!{command}

2021-09-23 16:40:57.906996: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 16:40:57.915713: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 16:40:57.916302: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
W0923 16:40:57.922617 140015066154880 model_lib_v2.py:1082] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0923 16:40:57.922858 140015066154880 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat

#9. Freeze graph

In [249]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [250]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

In [251]:
print(command)

python Tensorflow/models/research/object_detection/exporter_main_v2.py  --input_type=image_tensor --pipeline_config_path=/content/drive/MyDrive/workspaces_tensorflow/test_continue_training/test/pipeline.config --trained_checkpoint_dir=/content/drive/MyDrive/workspaces_tensorflow/test_continue_training/test/checkpoint --output_directory=/content/drive/MyDrive/workspaces_tensorflow/test_continue_training/test/export


In [252]:
!{command}

2021-09-23 16:50:44.412189: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 16:50:44.421731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 16:50:44.422347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 16:50:44.431862: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-23 16:50:44.432470: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

# 10. Zip model and copy to GG Drive

In [ ]:
import datetime
import pytz

def shorten_time(date):
    dt_year = date[0:4]
    dt_month = date[5:7]
    dt_day = date[8:10]
    dt_hr = date[11:13]
    dt_min = date[14:16]
    dt_sec = date[17:19]
    dt_full = "{}{}{}_{}{}{}".format(dt_year,dt_month,dt_day,dt_hr,dt_min,dt_sec)
    return dt_full

start_date = str(datetime.datetime.now(pytz.timezone('Asia/Bangkok')))[:19]
start_date_short = shorten_time(start_date)
print(start_date)
print(start_date_short)

In [ ]:
%cd {paths['MODEL_PATH']}

In [ ]:
export_model_path = "next_trained_model"
export_model_zip = "{}_{}.zip".format(export_model_path,start_date_short)
print(export_model_path)
print(export_model_zip)

In [ ]:
!zip -r {export_model_zip} {export_model_path}

In [ ]:
%cd "/content"

# 11. Notification via Email

In [ ]:
import datetime
import pytz

finish_date = str(datetime.datetime.now(pytz.timezone('Asia/Bangkok')))[:19]
finish_date_short = shorten_time(finish_date)

print(finish_date)
print(finish_date_short)

In [ ]:
sd_year = int(start_date_short[0:4])
sd_month = int(start_date_short[4:6])
sd_day = int(start_date_short[6:8])
sd_hour = int(start_date_short[9:11])
sd_min = int(start_date_short[11:13])
sd_sec = int(start_date_short[13:15])

fd_year = int(finish_date_short[0:4])
fd_month = int(finish_date_short[4:6])
fd_day = int(finish_date_short[6:8])
fd_hour = int(finish_date_short[9:11])
fd_min = int(finish_date_short[11:13])
fd_sec = int(finish_date_short[13:15])

In [ ]:
start = datetime.datetime(year=sd_year, month=sd_month, day=sd_day, hour=sd_hour, minute=sd_min,second=sd_sec)
end = datetime.datetime(year=fd_year, month=fd_month, day=fd_day, hour=fd_hour, minute=fd_min,second=fd_sec)
diff = end - start
print(diff)

In [ ]:
import smtplib
from email.message import EmailMessage

msg_subject = "[GG Colab] Model {}".format(start_date_short)
msg_content = "It take {} for training.".format(diff)

msg = EmailMessage()
msg['Subject'] = msg_subject
msg['From'] = '*'
msg['To']='ptmhoang97@gmail.com'
msg.set_content(msg_content)

server = smtplib.SMTP_SSL('smtp.gmail.com',465)
server.login("ptmhoang97@gmail.com","euclxykrqkkrquqz")
server.send_message(msg)
server.quit()